In [ ]:
import aiohttp
import asyncio


class Flashlight:
    COMMANDS = {"on": 'turn_on', "off": 'turn_off', "color": 'change_color'}
    COLORS = {1: "white", 2: "red", 3: "blue", 4: "green"}

    def __init__(self, status=0, color=1):
        self.status = status
        self.color = self.COLORS[color]

    async def command_accept(self, command, metadata=None):
        if command in self.COMMANDS:
            method_name = self.COMMANDS[command]
            method = getattr(self, method_name)
            method(metadata)
        else:
            pass

    def turn_on(self, metadata):
        if self.status == 0:
            self.status = 1
            print(f"Фонарь включен, установлен цвет: {self.color}")
        else:
            print("Фонарь уже включен")

    def turn_off(self, metadata):
        if self.status == 1:
            self.status = 0
            print("Фонарь выключен")
        else:
            print("Фонарь не включен")

    def change_color(self, metadata):
        if self.status == 1 and self.color != self.COLORS[metadata]:
            self.color = self.COLORS[metadata]
            print(f"Выставлен {self.color} цвет")
        elif self.status != self.COLORS[metadata]:
            print(f"Цвет {self.color} уже выставлен")
        else:
            print("Фонарь не включен")


async def processing_messages(message, flashlight):
    command = message.get("command").lower()
    metadata = message.get("metadata")
    await flashlight.command_accept(command, metadata)


async def main():
    while True:
        host = input("input address ") or "127.0.0.1"
        port = int(input("input port ") or 9999)
        if not (0 < port < 65535):
            raise ValueError
        break

    flashlight = Flashlight()
    session = aiohttp.ClientSession()
    while True:
        async with session.ws_connect(f'ws://{host}:{port}') as ws:
            await ws.send_str(input("Input message "))
            async for msg in ws:
                if msg.type == aiohttp.WSMsgType.TEXT:
                    message = msg.json()
                    await processing_messages(message, flashlight)
                elif msg.type == aiohttp.WSMsgType.ERROR:
                    print(f'Подключение к веб-сокету завершилось ошибкой: {ws.exception()}')
                    break



asyncio.run(main())